In [ ]:
# download imageNet class label
!wget https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json

In [ ]:
import json

from PIL import Image
import torch
from torch.nn import functional as F
from torchvision import models, transforms

In [ ]:
# prepare label name
with open("imagenet_class_index.json") as f:
    class_idx = json.load(f)
    # 0 -> 'tench'
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
    # 'n01440764' -> 'tench
    cls2label = {class_idx[str(k)][0]: class_idx[str(k)][1] for k in range(len(class_idx))}
    # 0 -> 'n01440764'
    idx2cls = [class_idx[str(k)][0] for k in range(len(class_idx))]

# prepare model
model = models.resnet50(pretrained=True)
model = model.eval()

# prepare transform
transform_i = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224)
])
transform_t = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
def predict(path):
    img = Image.open(path)
    img = transform_i(img)
    img_t = transform_t(img)

    with torch.no_grad():
        prob = F.softmax(model(img_t[None]), dim=1)[0]

    idx = prob.argmax().item()
    print('label "{}", class "{}"'.format(idx2label[idx], idx2cls[idx]))
    print('confidence {:.3f}'.format(prob[idx].item()))
    return img

In [ ]:
path = '/path/to/image'
predict(path)

References
* https://discuss.pytorch.org/t/imagenet-classes/4923